# Cartographie sous `python`

Dans ce TP, nous allons aborder l'aspect cartographie sous `python` avec le package `folium` (à installer donc). L'idée sera d'ajouter à un fond de carte des formes de couleurs, en fonction d'une information tierce.

## Création de cartes

Le package `folium` est une interface entre `python` et la librairie [`leaflet`](https://leafletjs.com/). Elle s'utilise très facilement, comme nous pouvons le voir ci-dessous pour la création d'une carte. Les coordonnées indiquées sont la latitude et la longitude de New-York.

In [1]:
import folium
centre = [40.71427, -74.00597]
newyork = folium.Map(location = centre, zoom_start = 12)

Pour obtenir la carte, il suffit d'appeler l'objet ainsi créé.

In [2]:
newyork

### Ajout de marqueurs

La fonction `Marker()` permet d'ajouter une marque sur la carte pour signifier un emplacement précis. Il est possible de spécifier un contenu à afficher (sous la forme d'un *pop-up*) lorsque l'utilisateur va cliquer sur le marqueur. Celui-ci sera placé à la localisation fournie en premier, au format `[latitude, longitude]`. Ce contenu est spécifié dans l'argument `popup` (ici, une simple chaîne de caractères, mais cela peut être plus élaboré). Il est aussi possible customiser l'icône du marqueur. Une fois ce marqueur créé, il faut l'ajouter à la carte, via la fonction `add_to()`.

In [3]:
folium.Marker(centre, popup = "New-York").add_to(newyork)
newyork

### Ajout de cercles

La fonction `CircleMarker()` permet elle de créer des cercles, en donnant la localisation du centre et le rayon (via le paramètre `radius`). Idem que pour `Marker()`, on peut spécifier le contenu de la *popup* affichée lors d'un clic, et il est nécessaire de l'ajouter à la carte. Il est possible aussi de spécifier la couleur du cercle (via `color`) et la couleur de remplissage (via `fillColor`).

In [4]:
folium.CircleMarker(centre, popup = "New-York", radius = 10).add_to(newyork)
newyork

## Informations géographiques

Pour pouvoir ajouter ces informations, nous allons importer les contours des 5 quartiers new-yorkais disponibles dans [ce fichier](newyork-borough.geojson). Les données à récupérer sont celles au format `GeoJSON`. Vous devez les sauvegarder, idéalement dans le même répertoire que votre *notebook*. 

In [5]:
import json
geo = json.load(open("newyork-borough.geojson"))

In [6]:
geo.keys()

dict_keys(['type', 'crs', 'features'])

Le format [GeoJSON](http://geojson.org/) est de plus en plus utilisé pour représenter des données de type géographique. Il suit des spécifications précises, et peut être utilisé directement comme nous le verrons plus loin.

Dans cet objet, transformé en dictionnaire sous `python`, il y a deux éléments : le type et les informations (nommées `features`)

In [7]:
geo.keys()

dict_keys(['type', 'crs', 'features'])

In [8]:
geo["type"]

'FeatureCollection'

Nous vérifions bien que nous avons les 5 quartiers de New-York dans l'objet `features`, qui est une `list`.

In [9]:
len(geo["features"])

5

Dans chaque objet de cette liste, nous disposons aussi de différents objets de type `Feature`. 

In [10]:
geo["features"][0].keys()

dict_keys(['type', 'properties', 'id', 'geometry'])

In [11]:
geo["features"][0]["id"]

0

In [12]:
geo["features"][0]['type']

'Feature'

Dans le champs `properties`, il y a différentes informations utiles, dont les coordonnées *longitude* et *latitude* du centre.

In [13]:
geo["features"][4]['properties']

{'BoroCode': 2,
 'BoroName': 'Bronx',
 'Shape_Area': 1186804144.79,
 'Shape_Leng': 464517.89055,
 'coordinates': [40.84478, -73.86483],
 'highlight': {},
 'style': {'color': 'black',
  'fillColor': '#fdc47a',
  'fillOpacity': 0.6,
  'opacity': 1,
  'weight': 1}}

Et dans la partie `geometry`, nous voyons que c'est un `MultiPolygon` qui est représenté, avec toutes les coordonnées des sommets de ce polygone.

In [14]:
geo["features"][0]['geometry']

{'coordinates': [[[[-74.24927399999999, 40.54492199999982],
    [-74.249211, 40.54506399999981],
    [-74.24741499999999, 40.549199999999814],
    [-74.23921099999998, 40.55376399999984],
    [-74.233052, 40.557608999999815],
    [-74.23278799999997, 40.55777299999983],
    [-74.23252499999998, 40.55793799999982],
    [-74.23215199999999, 40.55815799999982],
    [-74.23213399999999, 40.558169999999826],
    [-74.23192599999999, 40.55831099999982],
    [-74.23169299999998, 40.55845699999982],
    [-74.23099199999999, 40.5583959999998],
    [-74.23031199999998, 40.558335999999834],
    [-74.23029099999998, 40.55833399999982],
    [-74.22968499999999, 40.55828099999982],
    [-74.22907999999998, 40.558227999999815],
    [-74.22823999999999, 40.55813099999981],
    [-74.22739999999999, 40.55803399999982],
    [-74.22595599999998, 40.557867999999814],
    [-74.224513, 40.557700999999824],
    [-74.21839799999997, 40.55699599999982],
    [-74.21683899999998, 40.558617999999804],
    [-74.216

### Marqueur par quartier

Avec les informations contenues dans ce fichier, nous allons pouvoir ajouter un marqueur pour chaque quartier, en mettant dans la *popup*, son nom.

In [15]:
newyork_qu_marker = folium.Map(location = centre, zoom_start = 11)
for qu in geo["features"]:
    prop = qu["properties"]
    folium.Marker(prop["coordinates"], popup = prop["BoroName"]).add_to(newyork_qu_marker)
newyork_qu_marker

### Cercle par arrondissement

Toujours avec les mêmes informations, nous pouvons disposer des cercles pour chaque arrondissement. Ici, nous décidons de régler la taille des cercles (via `radius`) en fonction de la surface des arrondissements. Nous remarquons que nous devons faire cela manuellement (ici, on divise la surface par 1000000 pour avoir des tailles raisonnables).

In [16]:
newyork_qu_circle = folium.Map(location = centre, zoom_start = 11)
for qu in geo["features"]:
    prop = qu["properties"]
    folium.CircleMarker(prop["coordinates"], 
                        popup = prop["BoroName"],
                        radius = prop["Shape_Area"] / 1e08,
                        fill = True).add_to(newyork_qu_circle)
newyork_qu_circle

### Quartiers

Enfin, il est possible de faire des [cartes choroplètes](https://fr.wikipedia.org/wiki/Carte_choropl%C3%A8the), dans lesquelles les zones sont coloriées selon une mesure statistique.

Pour cela, nous devons devons utiliser la fonction `choropleth()` de la carte créée. Nous passons dans le paramètre `geo_data` les données contenues dans le fichier `GeoJSON` importées plus haut. 

In [17]:
newyork = folium.Map(location = centre, zoom_start = 10)
newyork.choropleth(geo_data = geo)
newyork

#### Avec une information tierce

Bien évidemment, nous désirons ajouter une couleur. Pour cela, nous avons besoin d'un `DataFrame` contenant deux variables :

- un identifiant commun avec les informations contenues dans le `GeoJSON`
- la variable statistique d'intérêt (pour cet exemple, ce sera la surface)

Pour le construire, nous allons récupérer le nom du quartier (item `BoroName` dans `properties`) et la surface (`Shape_Area` dans `properties`). Voici comment les récupérer.

In [18]:
[qu["properties"]["BoroName"] for qu in geo["features"]]

['Staten Island', 'Queens', 'Brooklyn', 'Manhattan', 'Bronx']

In [19]:
[qu["properties"]["Shape_Area"] for qu in geo["features"]]

[1623853249.91, 3049947236.73, 1959432236.83, 636442167.467, 1186804144.79]

Nous créons donc maintenant le `DataFrame` avec ces deux informations.

In [20]:
import pandas

df = pandas.DataFrame({
    # "BoroCode" : [qu["properties"]["BoroCode"] - 1 for qu in geo["features"]], 
    "Quartier": [qu["properties"]["BoroName"] for qu in geo["features"]], 
    "Surface" : [qu["properties"]["Shape_Area"] / 1e08 for qu in geo["features"]]
})
df

,Quartier,Surface
0,Staten Island,16.238532
1,Queens,30.499472
2,Brooklyn,19.594322
3,Manhattan,6.364422
4,Bronx,11.868041


Pour relier ces informations à la carte, toujours dans la fonction `choropleth()`, nous devons définir les éléments suivants :

- `geo_data` : les contours au format `GeoJSON`
- `key_on` : l'item dans le `GeoJSON` avec lequel nous ferons la jointure
- `data` : le `DataFrame` dans lequel nous avons les informations statistiques
- `columns` : les deux colonnes à prendre
    1. clé de jointure
    2. mesure statistique
- `fill_color` : la palette de couleurs à utiliser (provenant de [Color Brewer](http://colorbrewer2.org/))

Nous obtenons maintenant la carte suivante.

In [21]:
newyork = folium.Map(location = centre, zoom_start = 10)
newyork.choropleth(geo_data = geo, key_on = "feature.properties.BoroName",
                   data = df, columns = ["Quartier", "Surface"],
                   fill_color = 'Spectral',
                   legend_name = "Surface du quartier de New-York")
newyork

## Exercices

Nous allons utiliser une base de données des restaurants new-yorkais disponible dans [ce fichier](restaurants.bson) (**attention** : il fait environ 10 Mo).  Nous disposons de deux niveaux d'informations géographiques dans la base des restaurants new-yorkais :

- les géolocalisation de chaque restaurant
- les quartiers correspondants à chaque restaurant

Avec ces deux informations, nous allons donc pouvoir créer des cartes, soit avec des points pour chaque station, soit des formes pour les quartiers.  Sur cette base, vous devez faire les demandes suivantes :

1. Afficher un marqueur pour chaque restaurant, sur la carte de New-York
1. Ajouter à ces marqueurs une popup intégrant le nom du restaurant
1. Améliorer ces popups, en y ajoutant le quartier
1. Améliorer encore en mettant en gras le nom, en italique le quartier, et en ajoutant l'adresse, ainsi que le nombre d'évaluation pour chaque score
1. Afficher les quartiers sur la carte de New-York, avec une couleur dépandant du nombre de restaurants présents dans le quartier
1. Faire la même carte, mais avec une couleur en fonction du score moyen

Vous trouverez ci-dessous le code pour importer les données dans python.

In [22]:
import bson
donnees = bson.decode_all(open('restaurants.bson', 'rb').read())

In [23]:
donnees[:1]

[{'_id': ObjectId('58ac16d1a251358ee4ee87dd'),
  'address': {'building': '1007',
   'coord': [-73.856077, 40.848447],
   'street': 'Morris Park Ave',
   'zipcode': '10462'},
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
    'grade': 'A',
    'score': 2},
   {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
   {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
   {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
   {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'}]